In [6]:
import kagglehub

path = kagglehub.dataset_download("soumendraprasad/musical-instruments-sound-dataset")
print(path)

#TODO: powershell 기준 데이터셋 정리 (아래 과정 순서대로 실행)
# cd @path@   <- .cache/kagglehub 안에 데이터셋 다운받은 경로(위 path 변수 복붙)
# mv Metadata_Test.csv Test_submission/Test_submission
# mv Metadata_Train.csv Train_submission/Train_submission
# mv Test_submission/Test_submission ../..               
# mv Train_submission/Train_submission ../..
# cd ../..
# rmdir versions
# 이후 musical-instruments-sound-dataset 디렉토리를 vscode working directory로 옮기면 됩니다.


C:\Users\USER\.cache\kagglehub\datasets\soumendraprasad\musical-instruments-sound-dataset\versions\3


In [2]:
from datasets import load_dataset, Dataset
from datasets import config
import shutil
import os

train_meta = "./musical-instruments-sound-dataset/Train_submission/Metadata_Train.csv"
test_meta = "./musical-instruments-sound-dataset/Test_submission/Metadata_Test.csv"

train_audio_dir = "./musical-instruments-sound-dataset/Train_submission/"
test_audio_dir = "./musical-instruments-sound-dataset/Test_submission/"



# (굳이 할 필요 없음)datasets 캐시 디렉토리 flush
# cache_dir = config.HF_DATASETS_CACHE
# print(cache_dir)
# shutil.rmtree(cache_dir)
# print(f"Deleted cache at: {cache_dir}")



train_dataset = Dataset.from_csv(train_meta)
test_dataset = Dataset.from_csv(test_meta)

def add_audio_path(example, audio_dir):
    example["FilePath"] = os.path.join(audio_dir, example["FileName"])
    return example

train_dataset = train_dataset.map(lambda x: add_audio_path(x, train_audio_dir))
test_dataset = test_dataset.map(lambda x: add_audio_path(x, test_audio_dir))

print(train_dataset[0])

{'FileName': '1-E1-Major 00.wav', 'Class': 'Sound_Guitar', 'FilePath': './musical-instruments-sound-dataset/Train_submission/1-E1-Major 00.wav'}


In [3]:
from datasets import Audio

train_dataset = train_dataset.cast_column("FilePath", Audio())
test_dataset = test_dataset.cast_column("FilePath", Audio())
train_dataset[0]

{'FileName': 'AR_A_fret_0-20.wav',
 'Class': 'Sound_Guitar',
 'FilePath': {'path': './musical-instruments-sound-dataset/Train_submission/AR_A_fret_0-20.wav',
  'array': array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         -1.43051147e-06, -9.53674316e-07, -4.76837158e-07]),
  'sampling_rate': 44100}}

In [4]:
import pandas as pd

#TODO: 동석님이 만들어주신 text caption 파일 working dir로 이동
train_caption = pd.read_csv('./train_dataset_100_1203.csv')
train_caption.head()

,Unnamed: 0,audio_path,audio_caption
0,0,1-E1-Major 00.wav,[0:00-10:00]\nThis clip features an electric g...
1,1,1-E1-Major 01.wav,[0:00-10:00]\nThis song features an electric g...
2,2,1-E1-Major 02.wav,[0:00-10:00]\nThis song features an electric o...
3,3,1-E1-Major 03.wav,[0:00-10:00]\nThis music is instrumental. The ...
4,4,1-E1-Major 04.wav,[0:00-10:00]\nThis song features an electric g...


In [3]:
import librosa
import numpy as np
import torch

def audio_volumn_regularization(audio):
    basic_rms = 0.15
    rms = np.sqrt(np.mean(np.square(audio)))
    print(f'before audio reg(RMS): {rms}')
    audio *= basic_rms / rms
    return audio


def waveform_to_mel_spectrogram(y, original_sr):
    transform_sr=16000 #pipeline.vocoder.config.sampling_rate
    n_mels=64 # pipeline.vocoder.config.model_in_dim
    hop_length_ms=10
    hop_length = int(transform_sr * hop_length_ms / 1000) # must be 160 (equals to sr=16000, hop_length_ms=10)
    frame_length = 4096  # domain tradition
    n_fft = frame_length
    duration = 10.24
    fixed_n_frames = transform_sr * duration

    #y, _ = librosa.load(audio_path, sr=original_sr, duration=duration)
    y_resampled = librosa.resample(y, orig_sr=original_sr, target_sr=transform_sr)
    
    y_resampled_scaled = audio_volumn_regularization(y_resampled)
    
    mel_spectrogram = librosa.feature.melspectrogram(
        y=y_resampled_scaled,
        sr=transform_sr,
        n_fft=n_fft,
        hop_length=hop_length,
        win_length=frame_length,
        n_mels=n_mels,
    )

    log_mel_spectrogram = librosa.power_to_db(mel_spectrogram, ref=1.0)

    return torch.tensor(log_mel_spectrogram.T).unsqueeze(0)



# Preprocessing #
1) 10초 미만 오디오 제거
2) text caption csv에 없는 오디오 제거
3) 오디오 10초 단위로 slicing 

In [ ]:
import random
import numpy as np
import librosa
import torch
import pandas as pd
from datasets import Dataset


def filter_below_10_sec(example, slice_duration=10.24):
    audio = example["FilePath"]
    sampling_rate = audio["sampling_rate"]
    audio_array = audio["array"]
    
    slice_samples = int(sampling_rate * slice_duration)
    total_samples = len(audio_array)
    
    for start in range(0, total_samples, slice_samples):
        end = min(start + slice_samples, total_samples)
        sliced_audio = audio_array[start:end]
        if len(sliced_audio) < slice_samples:
            if start==0:  
                return False
            return True

train_dataset = train_dataset.filter(filter_below_10_sec)


def filter_no_matching_caption(example, slice_duration=10.24):
    audio = example["FilePath"]
    sampling_rate = audio["sampling_rate"]
    audio_array = audio["array"]
    
    slice_samples = int(sampling_rate * slice_duration)
    total_samples = len(audio_array)
    #print(f'array_shape: {audio_array.shape}')
    example['SlicesOfMelSpectrogram'] = []
    
    
    for start in range(0, total_samples, slice_samples):
        end = min(start + slice_samples, total_samples)
        sliced_audio = audio_array[start:end]
        # Zero-pad if slice is shorter than required
        if len(sliced_audio) < slice_samples:
            if start==0:  
                #print('below 10 sec')
                return False
            padding = slice_samples - len(sliced_audio)
            sliced_audio = np.pad(sliced_audio, (0, padding), mode="constant")
        
        #caption 추가
        i = int(start/slice_samples)
        pattern = f'\\[{i*10}:.*-{(i+1)*10}:.*\\]'
        file_name_mask = train_caption['audio_path'] == example['FileName']
        check_index = train_caption[file_name_mask]['audio_caption'].str.contains(pattern, na=False).index
        if len(check_index)==0:
            return False
        return True
    

train_dataset = train_dataset.filter(filter_no_matching_caption)

dict_for_sliced_mel = {'log_mel_spectrogram':[], 'text_caption':[], 'instrument_class':[]}

def slice_audio(example, dict_for_new_datasets, slice_duration=10.24):
    audio = example["FilePath"]
    sampling_rate = audio["sampling_rate"]
    audio_array = audio["array"]
    
    slice_samples = int(sampling_rate * slice_duration)
    total_samples = len(audio_array)
    #print(f'array_shape: {audio_array.shape}')
    example['SlicesOfMelSpectrogram'] = []
    
    for start in range(0, total_samples, slice_samples):
        end = min(start + slice_samples, total_samples)
        sliced_audio = audio_array[start:end]
        # Zero-pad if slice is shorter than required
        if len(sliced_audio) < slice_samples:
            if len(sliced_audio) < 0.7 * slice_samples:
                continue
            padding = slice_samples - len(sliced_audio)
            sliced_audio = np.pad(sliced_audio, (0, padding), mode="constant")
        
        #caption 추가
        i = int(start/slice_samples)
        file_name_mask = train_caption['audio_path'] == example['FileName']
        # pattern = f'\\[{i*10}:.*-{(i+1)*10}:.*\\]'
        # check_index = train_caption[file_name_mask]['audio_caption'].str.contains(pattern, na=False).index
        # caption = train_caption['audio_caption'].loc[check_index[0]]
        
        caption = train_caption[file_name_mask]['audio_caption'].iloc[0]
        start_pattern = f'[{i*10}:00-'
        end_pattern = f'[{(i+1)*10}:00-'
        
        print(f'start pattern: {start_pattern}, end pattern: {end_pattern}')
        first_idx = caption.find(start_pattern)
        start_idx = caption[first_idx:].find(']')+first_idx+1
        end_idx = caption.find(end_pattern)
        if (first_idx == -1 and end_idx == -1):
            print(f'no pattern matched: {caption}')
            continue
        elif end_idx == -1:
            caption = caption[start_idx:]
        else:
            caption = caption[start_idx:end_idx]

        print(f'caption matched: {caption}')
        
        log_mel_spectrogram = waveform_to_mel_spectrogram(sliced_audio, sampling_rate)
        
        print(f'shape of log_mel_spectrogram: {log_mel_spectrogram.shape}') # [1, sequence_length, n_mels(=model_in_dim)] = [1, 1024, 64]
        
        dict_for_new_datasets['instrument_class'].append(example['Class'])
        dict_for_new_datasets['text_caption'].append(caption)
        dict_for_new_datasets['log_mel_spectrogram'].append(log_mel_spectrogram)

    return example



sliced_dataset = train_dataset.map(lambda x: slice_audio(x, dict_for_sliced_mel))
dict_for_sliced_mel

Map:   0%|          | 0/241 [00:00<?, ? examples/s]

start pattern: [0:00-, end pattern: [10:00-
caption matched: 
The low quality recording features a resonating piano melody. It sounds like a tutorial and the recording is noisy and in mono. 
 

torch.Size([1, 1025, 64])
start pattern: [10:00-, end pattern: [20:00-
caption matched: 
This is a low quality recording of a veena being played. This is a tutorial on how to tune a ukulele. The first half of the song is a lower note and it gives off happy and fun vibes. There are no other instruments in this song. 
 

torch.Size([1, 1025, 64])
start pattern: [0:00-, end pattern: [10:00-
caption matched: 
The low quality recording features a ukulele melody. It sounds like a tutorial and the recording is noisy and in mono. 
 

torch.Size([1, 1025, 64])
start pattern: [0:00-, end pattern: [10:00-
caption matched: 
This is a low quality recording of a veena being played. This is a classical Hindustani song. The veena plays a note bending the string to a higher note and then descending back to the o

{'log_mel_spectrogram': [tensor([[[-80.0000, -80.0000, -80.0000,  ..., -80.0000, -80.0000, -80.0000],
           [-80.0000, -80.0000, -80.0000,  ..., -80.0000, -80.0000, -80.0000],
           [-80.0000, -80.0000, -80.0000,  ..., -80.0000, -80.0000, -80.0000],
           ...,
           [-41.4414, -34.2239,  -5.9937,  ..., -60.2604, -59.8263, -59.5847],
           [-33.8540, -25.5225,  -6.6243,  ..., -52.2588, -51.8421, -51.6354],
           [-31.6688, -23.4319,  -8.8492,  ..., -50.0905, -49.6835, -49.4948]]],
         dtype=torch.float64),
  tensor([[[-39.8063, -31.4982, -17.5898,  ..., -65.8198, -66.6829, -68.2412],
           [-41.6452, -33.1229, -15.0386,  ..., -67.9834, -68.8013, -70.2193],
           [-48.7319, -40.6449, -14.2934,  ..., -75.9758, -76.7118, -77.8748],
           ...,
           [-40.5207, -39.5493, -37.8685,  ..., -60.1601, -59.7355, -59.5242],
           [-31.9951, -31.0932, -29.5223,  ..., -52.1576, -51.7507, -51.5731],
           [-29.5672, -28.6957, -27.1524,  

# Save seperate datasets #
악기 별로 구분하여 10초 단위로 slicing된 ['instrument_class', 'text_caption', 'log_mel_spectrogram'] datasets를 working dir에 저장.


해당 데이터셋은 LoRA 학습에서 사용하게 됩니다

In [7]:
preprocessed_dataset = Dataset.from_dict(dict_for_sliced_mel)
inst_class = list(set(dict_for_sliced_mel['instrument_class']))
for inst in inst_class:
    inst_preprocessed_dataset = preprocessed_dataset.filter(lambda x: x['instrument_class'] == inst)
    folder_name_of_preprocessed_dataset = f'{inst}_train_preprocessed_dataset'
    inst_preprocessed_dataset.save_to_disk(folder_name_of_preprocessed_dataset)
    print(f"Dataset saved to [{folder_name_of_preprocessed_dataset}] directory!")
    print(inst_preprocessed_dataset)

Filter:   0%|          | 0/478 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/161 [00:00<?, ? examples/s]

Dataset saved to [Sound_Piano_train_preprocessed_dataset] directory!
Dataset({
    features: ['log_mel_spectrogram', 'text_caption', 'instrument_class'],
    num_rows: 161
})


Filter:   0%|          | 0/478 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/128 [00:00<?, ? examples/s]

...

Saving the dataset (0/1 shards):   0%|          | 0/61 [00:00<?, ? examples/s]

Dataset saved to [Sound_Guitar_train_preprocessed_dataset] directory!
Dataset({
    features: ['log_mel_spectrogram', 'text_caption', 'instrument_class'],
    num_rows: 61
})


Filter:   0%|          | 0/478 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/128 [00:00<?, ? examples/s]

Dataset saved to [Sound_Violin_train_preprocessed_dataset] directory!
Dataset({
    features: ['log_mel_spectrogram', 'text_caption', 'instrument_class'],
    num_rows: 128
})
